<a href="https://colab.research.google.com/github/harshraj7795/Fake-News-Identification-using-LSTM/blob/master/Fake_news_withdropout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install keras

In [3]:
#loading libraries
import numpy as np
import pandas as pd

In [4]:
#loading the dataframe
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dataset/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
#extracting the dependent and independent features
df = df.dropna()
x = df.drop('label',axis = 1)
y = df['label']

In [6]:
#importing libraries for LSTM modelling
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [7]:
#initializing vocabulary size
vocab = 5000

In [8]:
#importing libraries for text processing
from tensorflow.keras.preprocessing.text import one_hot
import nltk
nltk.download('stopwords')
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
#text preprocessing
msg = x.copy()
msg.reset_index(inplace=True)

ps = PorterStemmer()
corpus = []

for i in range(len(msg)):
    newmsg = re.sub('[^a-zA-Z]',' ',msg['title'][i])
    newmsg = newmsg.lower()
    newmsg = newmsg.split()
    newmsg = [ps.stem(word) for word in newmsg if word not in stopwords.words('english')]
    newmsg = ' '.join(newmsg)
    corpus.append(newmsg)

In [10]:
#onehot representation
onehot = [one_hot(word,vocab) for word in corpus]

In [11]:
#embedded representations
sent_len = 20
emb_doc = pad_sequences(onehot,padding = 'pre', maxlen = sent_len)


In [12]:
## Creating model with dropout to prevent overfit
from tensorflow.keras.layers import Dropout
emb_feat_vectors = 40
model = Sequential()
model.add(Embedding(vocab,emb_feat_vectors,input_length=sent_len))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [13]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
#preparing the model for training the model
import numpy as np
x_new = np.array(emb_doc)
y_new = np.array(y)

from sklearn.model_selection import train_test_split
x_tr,x_ts,y_tr,y_ts = train_test_split(x_new,y_new,test_size = 0.3, random_state = 42)

In [15]:
#training the model
model.fit(x_tr,y_tr, validation_data=(x_ts,y_ts),epochs=10,batch_size=64)

Epoch 1/10
200/200 [==============================] - 35s 12ms/step - loss: 0.4833 - accuracy: 0.7353 - val_loss: 0.2003 - val_accuracy: 0.9132
Epoch 2/10
200/200 [==============================] - 2s 8ms/step - loss: 0.1356 - accuracy: 0.9456 - val_loss: 0.1961 - val_accuracy: 0.9183
Epoch 3/10
200/200 [==============================] - 2s 9ms/step - loss: 0.0908 - accuracy: 0.9666 - val_loss: 0.2104 - val_accuracy: 0.9200
Epoch 4/10
200/200 [==============================] - 2s 8ms/step - loss: 0.0647 - accuracy: 0.9789 - val_loss: 0.2403 - val_accuracy: 0.9156
Epoch 5/10
200/200 [==============================] - 2s 8ms/step - loss: 0.0437 - accuracy: 0.9862 - val_loss: 0.3197 - val_accuracy: 0.9185
Epoch 6/10
200/200 [==============================] - 2s 8ms/step - loss: 0.0280 - accuracy: 0.9916 - val_loss: 0.3812 - val_accuracy: 0.9132
Epoch 7/10
200/200 [==============================] - 2s 8ms/step - loss: 0.0164 - accuracy: 0.9950 - val_loss: 0.4206 - val_accuracy: 0.9079
Epoc

In [16]:
#evaluating the model
from sklearn.metrics import accuracy_score
y_pred = model.predict_classes(x_ts)
print(accuracy_score(y_ts,y_pred))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.907036091870215
